In [1]:
import argparse
import numpy as np
from time import time
import numpy as np
import random 
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.metrics import f1_score, roc_auc_score
import heapq
import pysnooper

#from imp import reload
#reload(data_loader)  #重新载入

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

parser = argparse.ArgumentParser()
# movie
parser.add_argument('--model_path', nargs='?', default='',help='Store model path.')
parser.add_argument('--dataset', type=str, default='music', help='which dataset to use')
parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
parser.add_argument('--n_epochs', type=int, default=30, help='the number of epochs')
parser.add_argument('--neighbor_sample_size', type=int, default=1, help='the number of neighbors to be sampled')
parser.add_argument('--emb_dim', type=int, default=64, help='dimension of user and entity embeddings')
parser.add_argument('--kge_dim', type=int, default=64, help='dimension of user and KG embeddings')
parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
parser.add_argument('--l2_weight', type=float, default=1e-7, help='weight of l2 regularization')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')
parser.add_argument('--seed', type=int, default=111, help='random seed')
parser.add_argument('--regs', nargs='?', default='[1e-5,1e-5,1e-2]',help='Regularization for user and item embeddings.')

parser.add_argument('--batch_size_cf', type=int, default=2048,help='CF batch size.')
parser.add_argument('--batch_size_kg', type=int, default=2048,help='KG batch size.')

parser.add_argument('--use_kge', type=bool, default=True,help='whether using knowledge graph embedding')
parser.add_argument('--use_att', type=bool, default=True,help='whether using attention mechanism')   
parser.add_argument('--verbose', type=int, default=1,help='Interval of evaluation.')
parser.add_argument('--dropout', type=float, default=0.1, help='keep probability(1-dropout)')
parser.add_argument('--mess_dropout', nargs='?', default='[0.1]',
                      help='Keep probability w.r.t. message dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')
parser.add_argument('--node_dropout', nargs='?', default='[0.1]',
                      help='Keep probability w.r.t. node dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.')


args = parser.parse_args(args=[])
tf.set_random_seed(args.seed)
np.random.seed(args.seed)
print(args)

Namespace(aggregator='sum', batch_size_cf=2048, batch_size_kg=2048, dataset='music', dropout=0.1, emb_dim=64, kge_dim=64, l2_weight=1e-07, lr=0.001, mess_dropout='[0.1]', model_path='', n_epochs=30, n_iter=2, neighbor_sample_size=1, node_dropout='[0.1]', ratio=1, regs='[1e-5,1e-5,1e-2]', seed=111, use_att=True, use_kge=True, verbose=1)


In [3]:
from abc import ABCMeta, abstractmethod

class Fusion(object):
    @pysnooper.snoop(prefix='fusion_init')
    def __init__(self,emb_dim,neighbor_sample_size):
        self.neighbor_sample_size = neighbor_sample_size
        self.emb_dim = emb_dim
        x = self.neighbor_sample_size
        y = self.emb_dim

    
    @pysnooper.snoop(prefix='mix')
    def _fusion_neighbor_entities(self, neighbor_entities, neighbor_relations):
        
        

        
        
        #pad_numb = self.n_relation - len(neighbor_relations)
        #neighbor_relations = np.pad(neighbor_relations,(0,3),'constant',constant_values(0,0))
        # 第一个维度(行）前面扩展0行，后面扩展3行，第二个维度（列)不变
        #d = np.pad(d,((0,3),(0,0)),'constant',constant_values=(0,0))


        #  [batch_size,  n_neighbor, dim] * [batch_size,  n_neighbor, dim]--> [batch_size, n_neighbor,1]---
        #user_relation_scores = tf.reduce_mean(neighbor_vectors * neighbor_relations, axis=-1)---
        
        #[batch_size,neighbor_sample_size,dim]--->[batch_size,neighbor_sample_size]
        #_relation_scores = tf.reduce_mean(neighbor_relations, axis=-1)
        _relation_scores = tf.matmul(tf.multiply(neighbor_relations,neighbor_entities), self.weights) + self.bias
        _relation_scores = tf.squeeze(_relation_scores, axis=2)
         # [batch_size,  neighbor_sample_size] --> [batch_size, neighbor_sample_size]
        _relation_scores_normalized = tf.nn.softmax(_relation_scores, dim=-1)

        # [batch_size, neighbor_sample_size] --> [batch_size, neighbor_sample_size, 1]
        _relation_scores_normalized = tf.expand_dims(_relation_scores_normalized, axis=-1)

        # 1.[batch_size, neighbor_sample_size, 1]   [batch_size,, neighbor_sample_size, dim]--> [batch_size, neighbor_sample_size, dim]
        # 2.[batch_size, neighbor_sample_size, dim] ---> [batch_size, dim]
        #neighbors_fusion = tf.reduce_mean(_relation_scores_normalized * neighbor_entities, axis=1)
        neighbors_fusion = _relation_scores_normalized * neighbor_entities

        conv1D = tf.keras.layers.Conv1D(self.emb_dim,self.neighbor_sample_size,self.neighbor_sample_size,padding='valid')
        neighbors_fusion = conv1D(neighbors_fusion)
        #conv1d = tf.keras.layers.Conv1D(1,1,1,padding='valid')
        #max_pool_1d = tf.keras.layers.MaxPooling1D(pool_size=10,strides=10,padding='valid')
        #neighbors_fusion = max_pool_1d(neighbors_fusion)
        neighbors_fusion = tf.squeeze(neighbors_fusion,axis=1)
        #neighbors_fusion = tf.nn.softmax(tf.reduce_sum(_relation_scores_normalized * neighbor_entities, axis=2),dim=1)
        #neighbors_fusion = tf.nn.softmax(_relation_scores_normalized * neighbor_entities,dim=1)
        #neighbors_fusion = tf.reduce_max(_relation_scores_normalized * neighbor_entities, axis=1)
        
        return neighbors_fusion

    def __call__(self,neighbor_entities, neighbor_relations,user_embeddings):
        new_embedding = self.fusion(neighbor_entities, neighbor_relations, user_embeddings)
        return new_embedding
    
    @abstractmethod
    def fusion(self,neighbor_entities, neighbor_relations,user_embeddings):
        # dimension:
        # neighbor_vectors: [batch_size, neighbor_sample_size, n_neighbor, dim]
        # neighbor_relations: [batch_size, -neighbor_sample_size, dim]
        pass

class SumFusion(Fusion):
    def __init__(self, emb_dim,neighbor_sample_size,dropout=0.1, act=tf.nn.relu):
        #self.dropout = dropout
        super(SumFusion, self).__init__(emb_dim,neighbor_sample_size)
        self.act = act
        self.dropout=dropout
        
        initializer = tf.keras.initializers.glorot_normal()
        self.weights = tf.get_variable(shape=[self.emb_dim, 1], initializer=tf.keras.initializers.glorot_normal(),name='weights1')
        self.bias = tf.get_variable(shape=[1], initializer=tf.zeros_initializer(), name='bias1')
        
        self.weights2 = tf.get_variable(shape=[self.emb_dim*2, self.emb_dim], initializer=tf.keras.initializers.glorot_normal(),name='weights2')
        self.bias2 = tf.get_variable(shape=[self.emb_dim], initializer=tf.zeros_initializer(), name='bias2')
    @pysnooper.snoop(prefix='sum')
    def fusion(self, neighbor_entities, neighbor_relations, user_embeddings):
        
        neighbors_fusion = self._fusion_neighbor_entities(neighbor_entities, neighbor_relations)

        #output = user_embeddings + neighbors_fusion
        
        output = tf.concat([user_embeddings, neighbors_fusion], axis=-1)
        #output = tf.nn.dropout(output, keep_prob=1-self.dropout)
        output = tf.matmul(output, self.weights2) + self.bias2
        
        # [batch_size, dim]
        #output = tf.reshape(output, [self.batch_size, self.dim])

        return self.act(output)   

In [4]:
from sklearn.metrics import f1_score, roc_auc_score
#@pysnooper.snoop(prefix='RGAT')
class RGAT(object):
    def __init__(self,data,args,pretrain_data):
        self._parse_args(data, args,pretrain_data)
        self._build_inputs()
        self.weights = self._build_weights()
        self._build_model_cf()
        self._build_model_kg()
        self._build_loss()
        
        self.items = data[1]
        
    def _parse_args(self,data, args,pretrain_data):
        #n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg,rd
        self.n_users = data[0]
        self.n_items = data[1]
        self.n_entities = data[2]
        self.n_relations = data[3]
        #self.train_data = data[4]
        #self.eval_data = data[5]
        #self.test_data = data[6]
        #self.kg = data[7]
        #self.rd = data[8]
        self.dropout=args.dropout
        self.kge_dim = args.kge_dim
        self.emb_dim = args.emb_dim
        
        self.batch_size_kg = args.batch_size_kg
        self.batch_size_cf = args.batch_size_cf
        
        self.regs = eval(args.regs)
        self.lr =args.lr
        
        self.pretrain_data = pretrain_data
        self.neighbor_sample_size = args.neighbor_sample_size
        self.fussion_class = SumFusion

        
    def _build_inputs(self):
        # placeholder definition
        self.users = tf.placeholder(tf.int32, shape=(None,))
        self.pos_items = tf.placeholder(tf.int32, shape=(None,))
        self.neg_items = tf.placeholder(tf.int32, shape=(None,))

        # for knowledge graph modeling (TransD)
        #self.A_values = tf.placeholder(tf.float32, shape=[len(self.all_v_list)], name='A_values')

        self.h = tf.placeholder(tf.int32, shape=[None], name='h')
        self.r = tf.placeholder(tf.int32, shape=[None], name='r')
        self.pos_t = tf.placeholder(tf.int32, shape=[None], name='pos_t')
        self.neg_t = tf.placeholder(tf.int32, shape=[None], name='neg_t')
        # 2 --> (end,relation)
        self.neighbors = tf.placeholder(tf.int32,shape=[None,self.neighbor_sample_size,2],name ='neighbours')
        # dropout: node dropout (adopted on the ego-networks);
        # message dropout (adopted on the convolution operations).
        self.node_dropout = tf.placeholder(tf.float32, shape=[None])
        self.mess_dropout = tf.placeholder(tf.float32, shape=[None])
    
        self.labels = tf.placeholder(dtype=tf.float32, shape=[None], name='labels')
        
    def _build_weights(self):
        all_weights = dict()
        initializer = tf.keras.initializers.glorot_normal()

        if self.pretrain_data is None:
            all_weights['user_embed'] = tf.Variable(initializer([self.n_users, self.emb_dim]), name='user_embed')
            all_weights['entity_embed'] = tf.Variable(initializer([self.n_entities, self.emb_dim]), name='entity_embed')
            print('using xavier initialization')
        else:
            all_weights['user_embed'] = tf.Variable(initial_value=self.pretrain_data['user_embed'], trainable=True,
                                                    name='user_embed', dtype=tf.float32)

            item_embed = self.pretrain_data['item_embed']
            other_embed = initializer([self.n_entities - self.n_items, self.emb_dim])

            all_weights['entity_embed'] = tf.Variable(initial_value=tf.concat([item_embed, other_embed], 0),
                                                      trainable=True, name='entity_embed', dtype=tf.float32)
            print('using pretrained initialization')

        all_weights['relation_embed'] = tf.Variable(initializer([self.n_relations, self.kge_dim]),
                                                    name='relation_embed')
        all_weights['trans_W'] = tf.Variable(initializer([self.n_relations, self.emb_dim, self.kge_dim]))
        
        self.cf_weights=tf.Variable(initializer([self.kge_dim*2,1]),name='cf_weights')
        self.cf_bias = tf.get_variable(shape=[1], initializer=tf.zeros_initializer(), name='cf_bias')
        return all_weights        
    
    def _build_model_cf(self):
        #self.ua_embeddings, self.ea_embeddings = self._create_graphsage_embed()

        self.u_e = tf.nn.embedding_lookup(self.weights['user_embed'], self.users)
        
        self.pos_i_e = tf.nn.embedding_lookup(self.weights['entity_embed'], self.pos_items)
        self.neg_i_e = tf.nn.embedding_lookup(self.weights['entity_embed'], self.neg_items)


        #加入邻居居信息
        self.u_e = self.fusion_neighbor()
        #///self.batch_predictions = tf.matmul(self.u_e, self.pos_i_e, transpose_a=False, transpose_b=True)
        '''
        user_item = tf.concat([self.u_e, self.pos_i_e], axis=-1)
        user_item = tf.nn.dropout(user_item, keep_prob=1-self.dropout) 
        self.batch_predictions = tf.matmul(user_item, self.cf_weights) + self.cf_bias
        '''
        
        self.batch_predictions = tf.reduce_sum(tf.multiply(self.u_e, self.pos_i_e), axis=1)
        self.batch_predictions_normalized = tf.sigmoid(self.batch_predictions)
    
    @pysnooper.snoop(prefix='model')
    def fusion_neighbor(self):
        #aggregators = []  # store all aggregators
        
        #neighbors = cdata.construct_neighbours(pos_items)
        #print(neighbors[:,:,0])
        #print(neighbors[:,:,1])
        k = self.neighbors
        k_shape = self.neighbors.shape
        shape = [self.batch_size_cf,1]
        #self.neighbour_e = tf.nn.embedding_lookup(self.weights['entity_embed'], tf.reshape(self.neighbors[:,:,0],shape))
        #self.neighbour_r = tf.nn.embedding_lookup(self.weights['relation_embed'], tf.reshape(self.neighbors[:,:,1],shape))
        self.neighbour_e = tf.nn.embedding_lookup(self.weights['entity_embed'], self.neighbors[:,:,0])
        self.neighbour_r = tf.nn.embedding_lookup(self.weights['relation_embed'],self.neighbors[:,:,1])
        
        #relation_vectors = [tf.nn.embedding_lookup(self.relation_emb_matrix, i) for i in relations]
        
        n_iter = 2
        #for i in range(n_iter):
            #if i == n_iter - 1:
                                                      #batch_size, head,dim, dropout, act, name
            #    aggregator = self.aggregator_class(self.batch_size_cf, self.emb_dim, act=tf.nn.tanh)
            #else:
        fusion = self.fussion_class(self.emb_dim,self.neighbor_sample_size)
        
        # (self, neighbor_vectors, neighbor_relations, user_embeddings): [batch_size,  n_neighbor, dim]
        new_embedding = fusion(
                                neighbor_entities=self.neighbour_e,
                                neighbor_relations=self.neighbour_r,
                                user_embeddings=self.u_e)
        #entity_vectors_next_iter.append(vector)
           
        #res = tf.reshape(vector, [self.batch_size, self.dim])

        return new_embedding #, aggregators        
    
    def _build_model_kg(self):
        self.h_e, self.r_e, self.pos_t_e, self.neg_t_e = self._trnasR(self.h, self.r, self.pos_t, self.neg_t)
        #self.A_kg_score = self._generate_transE_score(h=self.h, t=self.pos_t, r=self.r)
        #self.A_out = self._create_attentive_A_out()       
    
    def _trnasR(self, h, r, pos_t, neg_t):
        
        #embeddings = tf.concat([self.weights['user_embed'], self.weights['entity_embed']], axis=0)
        # shape (n_entities, 1,emb_dim)
        embeddings = tf.expand_dims(self.weights['entity_embed'], 1)

        # head & tail entity embeddings: batch_size *1 * emb_dim
        h_e = tf.nn.embedding_lookup(embeddings, h)
        pos_t_e = tf.nn.embedding_lookup(embeddings, pos_t)
        neg_t_e = tf.nn.embedding_lookup(embeddings, neg_t)

        # relation embeddings: batch_size * kge_dim
        r_e = tf.nn.embedding_lookup(self.weights['relation_embed'], r)

        # relation transform weights: batch_size * kge_dim * emb_dim
        trans_W = tf.nn.embedding_lookup(self.weights['trans_W'], r)
        
        #neighbor_relation=dict()
        #for i in r:
        #    neighbor_relation[i]

        '''
        (batch_size ,1, emb_dim)*(batch_size,emb_dim,kge_dim) -->(batch_size , 1 , kge_dim)
        (batch_size , 1 , kge_dim) -> (batch_size , kge_dim)
        '''
        h_e = tf.reshape(tf.matmul(h_e, trans_W), [-1, self.kge_dim])
        pos_t_e = tf.reshape(tf.matmul(pos_t_e, trans_W), [-1, self.kge_dim])
        neg_t_e = tf.reshape(tf.matmul(neg_t_e, trans_W), [-1, self.kge_dim])
        
        # Remove the l2 normalization terms
        # h_e = tf.math.l2_normalize(h_e, axis=1)
        # r_e = tf.math.l2_normalize(r_e, axis=1)
        # pos_t_e = tf.math.l2_normalize(pos_t_e, axis=1)
        # neg_t_e = tf.math.l2_normalize(neg_t_e, axis=1)

        return h_e, r_e, pos_t_e, neg_t_e


    def _build_loss(self):
        pos_scores = tf.reduce_mean(tf.multiply(self.u_e, self.pos_i_e), axis=1)
        neg_scores = tf.reduce_mean(tf.multiply(self.u_e, self.neg_i_e), axis=1)

        regularizer = tf.nn.l2_loss(self.u_e) + tf.nn.l2_loss(self.pos_i_e) + tf.nn.l2_loss(self.neg_i_e)
        regularizer = regularizer / self.batch_size_cf

        # Using the softplus as BPR loss to avoid the nan error.
        cf_loss = tf.reduce_mean(tf.nn.softplus(-(pos_scores - neg_scores)))
        # maxi = tf.log(tf.nn.sigmoid(pos_scores - neg_scores))
        # base_loss = tf.negative(tf.reduce_mean(maxi))

        self.cf_reg_loss = self.regs[0] * regularizer
        
        self.cf_loss = cf_loss+self.cf_reg_loss

        
        
        def _get_kg_score(h_e, r_e, t_e):
            kg_score = tf.reduce_mean(tf.square((h_e + r_e - t_e)), 1, keepdims=True)
            return kg_score

        pos_kg_score = _get_kg_score(self.h_e, self.r_e, self.pos_t_e)
        neg_kg_score = _get_kg_score(self.h_e, self.r_e, self.neg_t_e)
        
        # Using the softplus as BPR loss to avoid the nan error.
        kg_loss = tf.reduce_mean(tf.nn.softplus(-(neg_kg_score - pos_kg_score)))
        #kg_loss = tf.reduce_mean(tf.nn.softplus(-(pos_kg_score - neg_kg_score)))
        #maxi = tf.log(tf.nn.sigmoid(neg_kg_score - pos_kg_score))
        #kg_loss = tf.negative(tf.reduce_mean(maxi))


        kg_reg_loss = tf.nn.l2_loss(self.h_e) + tf.nn.l2_loss(self.r_e) + \
                      tf.nn.l2_loss(self.pos_t_e) + tf.nn.l2_loss(self.neg_t_e)
        kg_reg_loss = kg_reg_loss / self.batch_size_kg

        #self.kge_loss2 = kg_loss
        self.kg_reg_loss = self.regs[1] * kg_reg_loss
        self.kg_loss = kg_loss + self.kg_reg_loss

        # Optimization process.
        #self.kg_opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.kg_loss)    
        
        self.loss = 0.8*self.cf_loss + 0.2*self.kg_loss 
        #self.loss = self.cf_loss + self.cf_reg_loss

        # Optimization process.RMSPropOptimizer
        self._opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)

    def train(self, sess, feed_dict):
        #return sess.run([self.kg_opt, self.cf_opt,self.loss,self.cf_loss,self.kg_loss,self.kg_reg_loss], feed_dict)
        return sess.run([self._opt,self.loss,self.cf_loss,self.cf_reg_loss,self.kg_loss,self.kg_reg_loss], feed_dict)
    
    #@pysnooper.snoop(prefix='recallK')
    def recallK(self,y_true,y_pred,k):
        def getTopK(y_pred,k):
            topK=[]
            a=y_pred
            row_len = y_true.shape[0]
            col_len = self.items
            for i in range(row_len):
                b = heapq.nlargest(k,range(col_len),a[i].take)
                topK.append(b)
            return topK
        x=self.items
        topK = getTopK(y_pred,k)
        print(topK)

        T=[]
        for i in range(len(y_true)):
            b = np.argwhere(y[i]==1)
            b = np.squeeze(b)
            T.append(b)
        print(x)

        recall_k = []
        for i in range(len(topK)):
            #print(topK[i])
            #print(T[i])
            recall_ = len(set(topK[i])&set(T[i]))/len(T[i])
            recall_k.append(recall_)
            print(recall/len(topK))
            return np.array(recall_k)
    
    def eval_(self, sess, feed_dict):
        labels, scores = sess.run([self.labels, self.batch_predictions_normalized], feed_dict)
        #AUC
        auc = roc_auc_score(y_true=labels, y_score=scores)
        
        #recall@K
        #rk = self.recallK(labels, scores,3)
        #F1
        scores[scores >= 0.5] = 1
        scores[scores < 0.5] = 0
        f1 = f1_score(y_true=labels, y_pred=scores)
        
        return auc, f1

    
    def show(self):
        print(self.n_users,self.test_data,self.weights)
        
        

In [5]:
class Data(object):
    def __init__(self,args):
        self.args=args
        self.batch_size_cf = args.batch_size_cf
        self.batch_size_kg = args.batch_size_kg
        self.mess_dropout = args.mess_dropout
        
    #@pysnooper.snoop(prefix='ctr_eval')
    def ctr_eval(self,sess, model, data,flag):
        start = 0
        auc_list = []
        f1_list = []
        rk_list = []
        _batch = self.batch_size_cf//2
        while start + _batch <= data.shape[0]:
            auc, f1 = model.eval_(sess, self.get_test_feed_dict(model, data, start, start + _batch,flag))
            auc_list.append(auc)
            f1_list.append(f1)
            
            start += _batch
        return float(np.mean(auc_list)), float(np.mean(f1_list))
        
    #@pysnooper.snoop(prefix='get_test_feed_dict')
    def get_test_feed_dict(self,model, data, start, end,flag):
        neg_items = []
        users = data[start:end,0]
        pos_items = data[start:end,1]
        label = data[start:end,2]
        
        n_user=len(users)
        all_user = np.squeeze(np.tile(users,(2,1)).reshape(-1,2*n_user))
        
        for u in users:
            #pos_items += sample_pos_items_for_u(u, 1)
            neg_items += self.sample_neg_items_for_u(u, 1,flag)
        all_item = np.squeeze(np.vstack((pos_items,neg_items)).reshape(-1,2*n_user))
        all_label = np.pad(label,(0,n_user),'constant',constant_values=(0,0))
        
        neighbors = cdata.construct_neighbours(pos_items)
        all_neighbors = np.vstack((neighbors,neighbors)).reshape(-1,self.args.neighbor_sample_size,2)
        feed_dict = {
            model.users: all_user,
            model.pos_items: all_item,
            model.neg_items: neg_items,
            model.labels:all_label,
            model.neighbors:all_neighbors,
        }
        return feed_dict
    
    def sample_neg_items_for_u(self,u, num,flag):
        neg_items = []
        if(flag=='train'):
            _user_dict = self.train_user_dict
        if(flag=='eval'):
            _user_dict = self.eval_user_dict
        if(flag=='test'):
            _user_dict = self.test_user_dict
        while True:
            if len(neg_items) == num: break
            neg_i_id = np.random.randint(low=0, high=self.n_item,size=1)[0]

            if neg_i_id not in _user_dict[u] and neg_i_id not in neg_items:
                neg_items.append(neg_i_id)
        return neg_items

        
            
            
            
    def load_data(self):
        self.n_user,self.n_item, self.train_data, self.eval_data, self.test_data,self.exist_users,self.train_user_dict,self.test_user_dict,self.eval_user_dict,self.n_interaction = self.load_rating()
        #n_entity, n_relation, adj_entity, adj_relation = load_kg(args)
        self.n_entity, self.n_relation, self.kg,self.rd,self.triplet_number = self.load_kg()
        print('data loaded.')

        return self.n_user, self.n_item, self.n_entity, self.n_relation, self.train_data, self.eval_data, self.test_data, self.kg,self.rd,self.n_interaction,self.triplet_number

    #@pysnooper.snoop(prefix='load_kg')
    def load_rating(self):
        print('reading rating file ...')

        # reading rating file
        rating_file = './data/' + self.args.dataset + '/ratings_final'
        if os.path.exists(rating_file + '.npy'):
            rating_np = np.load(rating_file + '.npy')
        else:
            rating_np = np.loadtxt(rating_file + '.txt', dtype=np.int64)
            np.save(rating_file + '.npy', rating_np)

        n_user = len(set(rating_np[:, 0]))
        n_item = len(set(rating_np[:, 1]))
        
        n_interaction = len(rating_np)
        
        
        train_data, eval_data, test_data = self.dataset_split(rating_np)

        #exist_users=list(set(train_data[:,0]))
        all_users=list(set(rating_np[:,0]))
        
        # {userid:[pos_id...]}}
        train_user_dict=dict()
        for id in range(len(train_data)):
            u_id = train_data[id][0]
            i_id = train_data[id][1]
            if u_id not in train_user_dict.keys():
                train_user_dict[u_id] = []
            train_user_dict[u_id].append(i_id)
        
        test_user_dict=dict()
        for id in range(len(test_data)):
            u_id = test_data[id][0]
            i_id = test_data[id][1]
            if u_id not in test_user_dict.keys():
                test_user_dict[u_id] = []
            test_user_dict[u_id].append(i_id)        
            
        eval_user_dict=dict()
        for id in range(len(eval_data)):
            u_id = eval_data[id][0]
            i_id = eval_data[id][1]
            if u_id not in eval_user_dict.keys():
                eval_user_dict[u_id] = []
            eval_user_dict[u_id].append(i_id)        
            
        return n_user, n_item, train_data, eval_data, test_data,all_users,train_user_dict,test_user_dict,eval_user_dict,n_interaction


    def dataset_split(self,rating_np):
        print('splitting dataset ...')

        # train:eval:test = 6:2:2
        eval_ratio = 0.2
        test_ratio = 0.2
        n_ratings = rating_np.shape[0]

        eval_indices = np.random.choice(list(range(n_ratings)), size=int(n_ratings * eval_ratio), replace=False)
        left = set(range(n_ratings)) - set(eval_indices)
        test_indices = np.random.choice(list(left), size=int(n_ratings * test_ratio), replace=False)
        train_indices = list(left - set(test_indices))
        if self.args.ratio < 1:
            train_indices = np.random.choice(list(train_indices), size=int(len(train_indices) * self.args.ratio), replace=False)

        train_data = rating_np[train_indices]
        eval_data = rating_np[eval_indices]
        test_data = rating_np[test_indices]

        
        return train_data, eval_data, test_data


    def load_kg(self):
        print('reading KG file ...')

        # reading kg file
        kg_file = './data/' + self.args.dataset + '/kg_final'
        if os.path.exists(kg_file + '.npy'):
            kg_np = np.load(kg_file + '.npy')
        else:
            kg_np = np.loadtxt(kg_file + '.txt', dtype=np.int64)
            np.save(kg_file + '.npy', kg_np)

        n_entity = len(set(kg_np[:, 0]) | set(kg_np[:, 2]))
        n_relation = len(set(kg_np[:, 1]))
        
        triplet_number = len(kg_np)

        kg,rd = self.construct_kg(kg_np)
        #adj_entity, adj_relation = construct_adj(args, kg, n_entity)

        return n_entity, n_relation, kg,rd ,triplet_number


    def construct_kg(self,kg_np):
        print('constructing knowledge graph ...')
        kg = dict()
        rd = dict()
        for triple in kg_np:
            head = triple[0]
            relation = triple[1]
            tail = triple[2]
            # treat the KG as an undirected graph
            if head not in kg.keys():
                kg[head] = []
            kg[head].append((tail, relation))
            #if tail not in kg:
            #    kg[tail] = []
            #kg[tail].append((head, relation))
            if relation not in rd.keys():
                rd[relation]=[]
            rd[relation].append((head,tail))
        return kg,rd


    def generate_train_feed_dict(self, model,start,end):
        self.generate_cf_batch(start,end)
        self.generate_kg_batch()
        feed_dict = {
            model.h: self.batch_data['heads'],
            model.r: self.batch_data['relations'],
            model.pos_t: self.batch_data['pos_tails'],
            model.neg_t: self.batch_data['neg_tails'],

            model.users: self.batch_data['users'],
            model.pos_items: self.batch_data['pos_items'],
            model.neg_items: self.batch_data['neg_items'],

            model.neighbors: self.batch_data['neighbors'],
            model.mess_dropout: eval(self.args.mess_dropout),
            model.node_dropout: eval(self.args.node_dropout),
        }

        return feed_dict
    
   
    def generate_cf_batch(self,start,end):
        
        users, pos_items, neg_items,neighbors = self._generate_train_cf_batch2(start,end)

        self.batch_data = {}
        self.batch_data['users'] = users
        self.batch_data['pos_items'] = pos_items
        self.batch_data['neg_items'] = neg_items
        
        self.batch_data['heads'] = pos_items
        self.batch_data['neighbors'] = neighbors
    
    def generate_kg_batch(self):
        heads, relations, pos_tails, neg_tails = self._generate_kg_batch()
        self.batch_data['relations'] = relations
        self.batch_data['pos_tails'] = pos_tails
        self.batch_data['neg_tails'] = neg_tails
        
        
    
    def _generate_train_cf_batch2(self,start,end):
        def sample_pos_items_for_u(u, num):
            pos_items = self.train_user_dict[u]
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num: break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]

                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch

        def sample_neg_items_for_u(u, num):
            neg_items = []
            while True:
                if len(neg_items) == num: break
                neg_i_id = np.random.randint(low=0, high=self.n_item,size=1)[0]

                if neg_i_id not in self.train_user_dict[u] and neg_i_id not in neg_items:
                    neg_items.append(neg_i_id)
            return neg_items

        pos_items, neg_items = [], []
        users = self.train_data[start:end,0]
        pos_items = self.train_data[start:end,1]
        for u in users:
            #pos_items += sample_pos_items_for_u(u, 1)
            neg_items += sample_neg_items_for_u(u, 1)

        neighbors = cdata.construct_neighbours(pos_items)
        return users, pos_items, neg_items,neighbors
   
    def _generate_kg_batch(self):
        exist_heads = self.kg.keys()
        '''
        if self.batch_size_kg <= len(exist_heads):
            heads = random.sample(exist_heads, self.batch_size_kg)
        else:
            heads = [random.choice(exist_heads) for _ in range(self.batch_size_kg)]
        '''
        heads = self.batch_data['pos_items']

        def sample_pos_triples_for_h(h, num):
            pos_triples = self.kg[h]
            n_pos_triples = len(pos_triples)

            pos_rs, pos_ts = [], []
            while True:
                if len(pos_rs) == num: break
                pos_id = np.random.randint(low=0, high=n_pos_triples, size=1)[0]

                t = pos_triples[pos_id][0]
                r = pos_triples[pos_id][1]

                if r not in pos_rs and t not in pos_ts:
                    pos_rs.append(r)
                    pos_ts.append(t)
            return pos_rs, pos_ts

        def sample_neg_triples_for_h(h, r, num):
            neg_ts = []
            while True:
                if len(neg_ts) == num: break

                t = np.random.randint(low=0, high=self.n_entity, size=1)[0]
                if (t, r) not in self.kg[h] and t not in neg_ts:
                    neg_ts.append(t)
            return neg_ts

        pos_r_batch, pos_t_batch, neg_t_batch = [], [], []

        for h in heads:
            pos_rs, pos_ts = sample_pos_triples_for_h(h, 1)
            pos_r_batch += pos_rs
            pos_t_batch += pos_ts

            neg_ts = sample_neg_triples_for_h(h, pos_rs[0], 1)
            neg_t_batch += neg_ts

        return heads, pos_r_batch, pos_t_batch, neg_t_batch
        

    def construct_neighbours(self,pos_items):
        neighbor_sample_size=self.args.neighbor_sample_size
        head_items = np.unique(pos_items)
        end = np.zeros((len(pos_items),neighbor_sample_size,2))  # 2 (end relation)
        for headid in head_items:
            #print(headid)
            #print(kg[userid])
            elements = np.array(self.kg[headid])
            numbers = len(elements)
            if(numbers>=neighbor_sample_size):
                sampled_indices = np.random.choice(list(range(numbers)),size=neighbor_sample_size,replace=False)
            else:
                sampled_indices = np.random.choice(list(range(numbers)),size=neighbor_sample_size,replace=True)
            #print(sampled_indices)
            #print(elements[sampled_indices,:])

            idx = np.argwhere(pos_items == headid)
            idx = np.array(idx.reshape(1, -1).squeeze(0))

            end[idx,:] = elements[sampled_indices,:]
        #print(end.astype(int))
        return end.astype(int)

In [6]:
cdata = Data(args)
data =cdata.load_data()
# 0       1        2           3          4          5            6      7  8    9              10
#n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg,rd,n_interaction,triplet_number

#print(data)

#print (data)


reading rating file ...
splitting dataset ...
reading KG file ...
constructing knowledge graph ...
data loaded.


In [7]:
print("user:%d,item:%d,interactions:%d,entity:%d,relation:%d,triplets:%d"%(data[0],data[1],data[9],data[2],data[3],data[10]))
print(data[4].shape[0],data[5].shape[0],data[6].shape[0])
print(data[4].shape[0]+data[5].shape[0]+data[6].shape[0])
print(len(data[7]))
print(data[9])

user:1872,item:3846,interactions:21173,entity:9366,relation:60,triplets:15518
12705 4234 4234
21173
3846
21173


In [8]:
print(len(data))
k=data[7]
#print(k.keys())
#print(k[67])
#print(k[14967])
#print(len(k[14967]))
#print(len(k[15061]))
'''
for i in range(0,100):
    if(len(k[i])>1):
        print(i,len(k[i]))
'''

11


'\nfor i in range(0,100):\n    if(len(k[i])>1):\n        print(i,len(k[i]))\n'

In [9]:
users, pos_items, neg_items,neighbors= cdata._generate_train_cf_batch2(1,20)


In [10]:
model = RGAT(data,args,None)

using xavier initialization
Instructions for updating:
dim is deprecated, use axis instead


modelSource path:... <ipython-input-4-8f5002ba56da>
modelStarting var:.. self = <__main__.RGAT object at 0x7f620c07a320>
model07:35:07.162966 call       111     def fusion_neighbor(self):
model07:35:07.163289 line       117         k = self.neighbors
modelNew var:....... k = <tf.Tensor 'neighbours:0' shape=(?, 1, 2) dtype=int32>
model07:35:07.163367 line       118         k_shape = self.neighbors.shape
modelNew var:....... k_shape = TensorShape([Dimension(None), Dimension(1), Dimension(2)])
model07:35:07.163965 line       119         shape = [self.batch_size_cf,1]
modelNew var:....... shape = [2048, 1]
model07:35:07.164105 line       122         self.neighbour_e = tf.nn.embedding_lookup(self.weights['entity_embed'], self.neighbors[:,:,0])
model07:35:07.176076 line       123         self.neighbour_r = tf.nn.embedding_lookup(self.weights['relation_embed'],self.neighbors[:,:,1])
model07:35:07.183627 line       127         n_iter = 2
modelNew var:....... n_iter = 2
model07:35:07.183783 lin

In [11]:
config = tf.ConfigProto()
#os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu_id)
os.environ["CUDA_VISIBLE_DEVICES"] = '0'   #指定第一块GPU可用
#config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 程序最多只能占用指定gpu50%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
train_data = data[4]
eval_data = data[5]
test_data = data[6]
for epoch in range(args.n_epochs):
    t1 = time()
    loss, base_loss,cf_loss,cf_reg_loss, kge_loss, kge_reg_loss = 0., 0., 0., 0.,0.,0.
    #n_batch = cdata.n_train // args.batch_size_cf + 1

    """
    *********************************************************
    Alternative Training for KGAT:
    ... phase 1: to train the recommender.
    
    for idx in range(n_batch):
        btime= time()

        batch_data = data_generator.generate_train_batch()
        feed_dict = data_generator.generate_train_feed_dict(model, batch_data)

        _, batch_loss, batch_base_loss, batch_kge_loss, batch_reg_loss = model.train(sess, feed_dict=feed_dict)

        loss += batch_loss
        base_loss += batch_base_loss
        kge_loss += batch_kge_loss
        reg_loss += batch_reg_loss

    if np.isnan(loss) == True:
        print('ERROR: loss@phase1 is nan.')
        sys.exit()

    """
    """
    *********************************************************
    Alternative Training for KGAT:
    ... phase 2: to train the KGE method & update the attentive Laplacian matrix.
    """

    
    n_batch = len(train_data) // args.batch_size_cf + 1

    if args.use_kge is True:
        # using KGE method (knowledge graph embedding).
        batch_kg_loss =0.0
        batch_kg_reg_loss = 0.0
        for idx in range(n_batch):
            start = idx*args.batch_size_cf
            end = start+args.batch_size_cf
            feed_dict = cdata.generate_train_feed_dict(model,start,end) #train
            start = end
            #_,_, batch_loss,batch_cf_loss,batch_kg_loss,batch_kg_reg_loss = model.train(sess,feed_dict = feed_dict)
            _, batch_loss,batch_cf_loss,batch_cf_reg_loss,batch_kg_loss,batch_kg_reg_loss = model.train(sess,feed_dict = feed_dict)
            
            loss += batch_loss
            kge_loss += batch_kg_loss
            cf_loss += batch_cf_loss
            cf_reg_loss +=batch_cf_reg_loss
            kge_reg_loss += batch_kg_reg_loss

    #if args.use_att is True:
        # updating attentive laplacian matrix.
    #    model.update_attentive_A(sess)

    if np.isnan(loss) == True:
        print('ERROR: loss@phase2 is nan.')
        sys.exit()

    show_step = 10
    
    if (epoch + 1) % show_step != 0:
        if args.verbose > 0 and epoch % args.verbose == 0:
            
            perf_str = 'Epoch %d [%.1fs]: [all_loss:%.5f, cf_loss:%.5f, cf_reg_loss:%.5f, kge_loss:%.5f, kge_reg_loss %.5f ]' % (epoch, time() - t1,loss,cf_loss,cf_reg_loss, kge_loss,kge_reg_loss)
            print(perf_str)
            train_auc, train_f1 = cdata.ctr_eval(sess, model, train_data,'train')
            eval_auc, eval_f1 = cdata.ctr_eval(sess, model, eval_data,'eval')
            test_auc, test_f1 = cdata.ctr_eval(sess, model, test_data,'test')
            
            print('---epoch %d    train auc: %.4f  f1: %.4f    eval auc: %.4f  f1: %.4f    test auc: %.4f  f1: %.4f'
                  % (epoch, train_auc, train_f1, eval_auc, eval_f1, test_auc, test_f1))
            #print('---epoch %d    train auc: %.4f  f1: %.4f '
            #      % (epoch, train_auc, train_f1))
        continue
    



Epoch 0 [3.1s]: [all_loss:4.85197, cf_loss:4.85199, cf_reg_loss:0.00000, kge_loss:4.85188, kge_reg_loss 0.00003 ]
---epoch 0    train auc: 0.7021  f1: 0.6673    eval auc: 0.6865  f1: 0.6502    test auc: 0.6783  f1: 0.6477
Epoch 1 [1.6s]: [all_loss:4.85160, cf_loss:4.85179, cf_reg_loss:0.00000, kge_loss:4.85088, kge_reg_loss 0.00003 ]
---epoch 1    train auc: 0.8076  f1: 0.7341    eval auc: 0.7768  f1: 0.7083    test auc: 0.7673  f1: 0.7023
Epoch 2 [1.5s]: [all_loss:4.85064, cf_loss:4.85128, cf_reg_loss:0.00000, kge_loss:4.84809, kge_reg_loss 0.00004 ]
---epoch 2    train auc: 0.8448  f1: 0.7663    eval auc: 0.8018  f1: 0.7265    test auc: 0.7958  f1: 0.7208
Epoch 3 [1.5s]: [all_loss:4.84846, cf_loss:4.84998, cf_reg_loss:0.00001, kge_loss:4.84235, kge_reg_loss 0.00004 ]
---epoch 3    train auc: 0.8534  f1: 0.7783    eval auc: 0.8103  f1: 0.7405    test auc: 0.7992  f1: 0.7338
Epoch 4 [1.5s]: [all_loss:4.84371, cf_loss:4.84663, cf_reg_loss:0.00003, kge_loss:4.83206, kge_reg_loss 0.00004 